In [1]:
import os
import streamlit as st 
import tqdm as notebook_tqdm
import pickle
import time
from dotenv import load_dotenv # Recommended for managing API keys securely

# Openai specific integrations
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

# Retrival QA
from langchain_classic.chains import RetrievalQAWithSourcesChain

# Text splitting 
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Document loaders and vector stores 
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_community.vectorstores import FAISS


e:\langchain_finance_news_research_tool_project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load environment variables from .env file
load_dotenv()

#or you can use this to check your key
gemini_api_key = os.getenv("GEMINI_API_KEY")

**Load data** 

In [3]:
loaders = UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
])

data = loaders.load()


In [4]:
len(data)

2

**Split data to create chunks**

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)

In [6]:
len(docs)

18

In [7]:
docs

[Document(metadata={'source': 'https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html'}, page_content='English\n\nHindi\n\nGujarati\n\nSpecials\n\nHello, Login\n\nHello, Login\n\nLog-inor Sign-Up\n\nMy Account\n\nMy Profile\n\nMy Portfolio\n\nMy Watchlist\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nMy Profile\n\nMy PRO\n\nMy Portfolio\n\nMy Watchlist\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nLogout\n\nLoans up to ₹50 LAKHS\n\nFixed Deposits\n\nCredit CardsLifetime Free\n\nCredit Score\n\nChat with Us\n\nDownload App\n\nFollow us on:\n\nNetwork 18\n\nGo Ad-Free\n\nMy Alerts\n\n>->MC_ENG_DESKTOP/MC_ENG_NEWS/MC_ENG_MARKETS_AS/MC_ENG_ROS_NWS_MKTS_AS_ATF_728\n\nMoneycontrol\n\nGo PRO NowPRO\n\nMoneycontrol PRO\n\nAdvertisement\n\nRemove Ad\n\nBusiness\n\nMarkets\n\nStocks\n\nEconomy\n\nCompanies\n\nTrends\n\nIPO\n\nOpinion\n\nEV Special\n\nHomeNewsBusinessMarketsWall Street rises as Tesla soars on AI optimism\n\nTrending Topi

In [8]:
# LLM Setup
# Using Gemini 2.5 Flash, excellent for RAG speed and performance
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    temperature=0.1, 
    google_api_key=gemini_api_key  # <-- PASS THE KEY HERE
)

**Create embeddings for these chunks and save them to FAISS index**

In [9]:
# Create the embeddings of the chunks using Gemini Embeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004",
    google_api_key=gemini_api_key  
)
# Pass the documents and embeddings inorder to create FAISS vector index
vectorIndex = FAISS.from_documents(docs, embeddings)

In [10]:
index_path = "faiss_index_gemini"

# 1. SAVE the index (uses FAISS's built-in, safe serialization method)
vectorIndex.save_local(folder_path=index_path)
print(f"FAISS index saved successfully to folder: {index_path}")

# 2. LOAD the index (This is how you would reuse the index later)
if os.path.exists(index_path):
    # CRITICAL: You must pass the embeddings object when loading FAISS
    vectorIndex = FAISS.load_local(
        folder_path=index_path, 
        embeddings=embeddings,
        allow_dangerous_deserialization=True # allow_dangerous_deserialization=True # Only if needed
        # Setting this to True might be necessary depending on your FAISS version
        # allow_dangerous_deserialization=True 
    )
    print(f"FAISS index loaded successfully from folder: {index_path}")

FAISS index saved successfully to folder: faiss_index_gemini
FAISS index loaded successfully from folder: faiss_index_gemini


**Retrieve similar embeddings for a given question and call LLM to retrieve final answer**

In [11]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())
chain

RetrievalQAWithSourcesChain(verbose=False, combine_documents_chain=MapReduceDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question.\nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=ChatGoogleGenerativeAI(profile={'max_input_tokens': 1048576, 'max_output_tokens': 65536, 'image_inputs': True, 'audio_inputs': True, 'pdf_inputs': True, 'video_inputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'image_tool_message': True, 'tool_choice': True}, model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), temperature=0.1, client=<google.ai.generativelanguage_v1beta.services.g

In [12]:
query = "what is the price of Tiago iCNG?"
# query = "what are the main features of punch iCNG?"
print(f"Running query: {query}")

result = chain.invoke({"question": query}, return_only_outputs=True)
print(result)

Running query: what is the price of Tiago iCNG?
{'answer': 'The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh.\n', 'sources': 'https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html'}
